In [1]:
#多台机器，每台机器有一个显卡、或者多个显卡，这种训练叫做分布式训练  
import  tensorflow as tf  

/usr/lib64/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#现在假设我们有A、B、C、D四台机器，首先需要在各台机器上写一份代码，并跑起来，各机器上的代码内容大部分相同  
# ，除了开始定义的时候，需要各自指定该台机器的task之外。以机器A为例子，A机器上的代码如下：  
cluster=tf.train.ClusterSpec({  
    "worker": [  
        "localhost:8880",#格式 IP地址：端口号，第一台机器A的IP地址 ,在代码中需要用这台机器计算的时候，就要定义：/job:worker/task:0  
        "localhost:8881"#第二台机器的IP地址 /job:worker/task:1  
    ],  
    "ps": [  
        "localhost:8882",#第四台机器的IP地址 对应到代码块：/job:ps/task:0  
    ]})  

In [3]:
server=tf.train.Server(cluster,job_name='worker',task_index=0)#找到‘worker’名字下的，task0，也就是机器A

In [4]:
with tf.device('/job:ps/task:0'):#参数定义在机器D上  
    w=tf.get_variable('w',(2,2),tf.float32,initializer=tf.constant_initializer(2))  
    b=tf.get_variable('b',(2,2),tf.float32,initializer=tf.constant_initializer(5))  
  
with tf.device('/job:worker/task:0/cpu:0'):#在机器A cpu上运行  
    addwb=w+b  
with tf.device('/job:worker/task:1/cpu:0'):#在机器B cpu上运行  
    mutwb=w*b  

In [ ]:
with tf.device(tf.train.replica_device_setter(worker_device='/job:worker/task:indexi',cluster=cluster)):